# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
# Your code here
from config import yelp_api_key
import requests
import json

In [14]:
import pandas as pd

In [38]:
headers = {'Authorization': f'Bearer {yelp_api_key}'}
url_params = {
                'term': 'pizza'.replace(' ', '+'),
                'location': 'NYC'.replace(' ', '+'),
                'limit': 50,
                'offset': 0
            }
resp = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params = url_params)

In [39]:
print(resp)

<Response [200]>


In [40]:
resp.json()

{'businesses': [{'id': 'ysqgdbSrezXgVwER2kQWKA',
   'alias': 'julianas-pizza-brooklyn-5',
   'name': "Juliana's Pizza",
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/clscwgOF9_Ecq-Rwsq7jyQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/julianas-pizza-brooklyn-5?adjust_creative=clCXICX9m9GSlzv5cn3yIQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=clCXICX9m9GSlzv5cn3yIQ',
   'review_count': 2225,
   'categories': [{'alias': 'pizza', 'title': 'Pizza'}],
   'rating': 4.5,
   'coordinates': {'latitude': 40.7026153030093,
    'longitude': -73.9934159993549},
   'transactions': [],
   'price': '$$',
   'location': {'address1': '19 Old Fulton St',
    'address2': '',
    'address3': '',
    'city': 'Brooklyn',
    'zip_code': '11201',
    'country': 'US',
    'state': 'NY',
    'display_address': ['19 Old Fulton St', 'Brooklyn, NY 11201']},
   'phone': '+17185966700',
   'display_phone': '(718) 596-6700',
   'distance': 323.20506308227306},

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [12]:
# Your code here; use a function or loop to retrieve all the results from your original request
resp.json().keys()

dict_keys(['businesses', 'total', 'region'])

In [13]:
resp.json()['total']

11400

In [20]:
df1 = pd.DataFrame.from_dict(resp.json()['businesses'])

In [21]:
df1

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-pizza-brooklyn-5,Juliana's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/clscwg...,False,https://www.yelp.com/biz/julianas-pizza-brookl...,2225,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7026153030093, 'longitude': -7...",[],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,323.205063
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOy...,False,https://www.yelp.com/biz/prince-street-pizza-n...,3666,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.7229087705545, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1941.963817
2,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6116,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978
3,C8j0q4Ma_S5hBGuAI-aaww,di-fara-pizza-brooklyn,Di Fara Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,https://www.yelp.com/biz/di-fara-pizza-brookly...,3519,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.625093, 'longitude': -73.961531}","[pickup, delivery]",$$,"{'address1': '1424 Ave J', 'address2': '', 'ad...",+17182581367,(718) 258-1367,9352.269073
4,EDYTUNp4bFO4dAwJMGuwCQ,pizza-collective-new-york,Pizza Collective,https://s3-media1.fl.yelpcdn.com/bphoto/kDJkUk...,False,https://www.yelp.com/biz/pizza-collective-new-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.77795, 'longitude': -73.98155}","[pickup, delivery]",NaN,"{'address1': '2060 Broadway', 'address2': None...",+16466930949,(646) 693-0949,8090.017092
5,WG639VkTjmK5dzydd1BBJA,rubirosa-new-york-2,Rubirosa,https://s3-media4.fl.yelpcdn.com/bphoto/LuSzR8...,False,https://www.yelp.com/biz/rubirosa-new-york-2?a...,2265,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.5,"{'latitude': 40.722766, 'longitude': -73.996233}","[pickup, delivery]",$$,"{'address1': '235 Mulberry St', 'address2': ''...",+12129650500,(212) 965-0500,1932.946770
6,v1DHGRNCH9247WLYoaoA9A,l-industrie-pizzeria-brooklyn,L'industrie Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/SRL_Rz...,False,https://www.yelp.com/biz/l-industrie-pizzeria-...,532,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.71162, 'longitude': -73.95783}","[pickup, delivery]",$,"{'address1': '254 S 2nd St', 'address2': '', '...",+17185990002,(718) 599-0002,3145.016041
7,2ggi1luerd3Mf_C82JvCYg,grimaldis-pizzeria-brooklyn-4,Grimaldi's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/KnocNR...,False,https://www.yelp.com/biz/grimaldis-pizzeria-br...,4613,"[{'alias': 'pizza', 'title': 'Pizza'}]",3.5,"{'latitude': 40.702583, 'longitude': -73.9932413}","[pickup, delivery]",$$,"{'address1': '1 Front St', 'address2': '', 'ad...",+17188584300,(718) 858-4300,330.347462
8,QR7eMOsOXxGg26xx4wQUOw,bar-dough-new-york,Bar Dough,https://s3-media1.fl.yelpcdn.com/bphoto/UBvxvJ...,False,https://www.yelp.com/biz/bar-dough-new-york?ad...,31,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 40.7603523698496, 'longitude': -7...","[pickup, delivery]",NaN,"{'address1': '350 W 46th St', 'address2': '', ...",+19172620543,(917) 262-0543,6116.732504
9,xxgcU_oFIox_pAmroo9WwA,front-street-pizza-brooklyn,Front Street Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/ybrfmg...,False,https://www.yelp.com/biz/front-street-pizza-br...,360,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.70244, 'longitude': -73.98943}","[pickup, delivery]",$,"{'address1': '80 Front St', 'address2': '', 

In [22]:
df1.append(df1)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-pizza-brooklyn-5,Juliana's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/clscwg...,False,https://www.yelp.com/biz/julianas-pizza-brookl...,2225,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7026153030093, 'longitude': -7...",[],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,323.205063
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOy...,False,https://www.yelp.com/biz/prince-street-pizza-n...,3666,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.7229087705545, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1941.963817
2,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6116,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978
3,C8j0q4Ma_S5hBGuAI-aaww,di-fara-pizza-brooklyn,Di Fara Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,https://www.yelp.com/biz/di-fara-pizza-brookly...,3519,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.625093, 'longitude': -73.961531}","[pickup, delivery]",$$,"{'address1': '1424 Ave J', 'address2': '', 'ad...",+17182581367,(718) 258-1367,9352.269073
4,EDYTUNp4bFO4dAwJMGuwCQ,pizza-collective-new-york,Pizza Collective,https://s3-media1.fl.yelpcdn.com/bphoto/kDJkUk...,False,https://www.yelp.com/biz/pizza-collective-new-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.77795, 'longitude': -73.98155}","[pickup, delivery]",NaN,"{'address1': '2060 Broadway', 'address2': None...",+16466930949,(646) 693-0949,8090.017092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,58YvTYsyxNyGixTAZiMHcg,williamsburg-pizza-new-york-3,Williamsburg Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/uMXAMU...,False,https://www.yelp.com/biz/williamsburg-pizza-ne...,228,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71822, 'longitude': -73.99123}","[pickup, delivery]",$,"{'address1': '277 Broome St', 'address2': '', ...",+12122264455,(212) 226-4455,1447.543509
46,atbANdSznV29GI066I2Ijw,champion-pizza-new-york-6,Champion Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/zDLR1A...,False,https://www.yelp.com/biz/champion-pizza-new-yo...,314,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7215270996094, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '17 Cleveland Pl', 'address2': No...",+12122263777,(212) 226-3777,1801.476444
47,15k7iqFbhf4h8L01yggLqg,song-e-napule-new-york,Song E Napule,https://s3-media2.fl.yelpcdn.com/bphoto/y9G-nh...,False,https://www.yelp.com/biz/song-e-napule-new-yor...,489,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.72813, 'longitude': -74.00188}","[pickup, delivery]",$$,"{'address1': '146 W Houston St', 'address2': N...",+12125331242,(212) 533-1242,2607.168271
48,pGr5QuQU7Ur5GHLvXbT8bw,paulie-gees-brooklyn,Paulie Gee's,https://s3-media2.fl.yelpcdn.com/bphoto/1j39J4...,False,https://www.yelp.com/biz/paulie-gees-brooklyn?...,1252,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.729546, 'longitude': -73.958568}",[],$$,"{'address1': '60 Greenpoint Ave', 'address2': ...",+13479873747,(347) 987-3747,4027.308251


In [95]:
def get_pizza_df(total):
    headers = {'Authorization': f'Bearer {yelp_api_key}'}
    url_params = {
                    'term': 'pizza'.replace(' ', '+'),
                    'location': 'NYC'.replace(' ', '+'),
                    'limit': 50
                }
    df = pd.DataFrame.from_dict(requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params = url_params).json()['businesses'])
    for n in range(1, int(total//50)):
        url_params['offset'] = n*50
        resp = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params = url_params)
        new_df = pd.DataFrame.from_dict(resp.json()['businesses'])
        df = pd.concat([df, new_df], ignore_index=True)
    return df

In [99]:
df_all = get_pizza_df(1000)

In [100]:
df_all

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ysqgdbSrezXgVwER2kQWKA,julianas-pizza-brooklyn-5,Juliana's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/clscwg...,False,https://www.yelp.com/biz/julianas-pizza-brookl...,2225,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.7026153030093, 'longitude': -7...",[],$$,"{'address1': '19 Old Fulton St', 'address2': '...",+17185966700,(718) 596-6700,323.205063
1,zj8Lq1T8KIC5zwFief15jg,prince-street-pizza-new-york-2,Prince Street Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/ZAukOy...,False,https://www.yelp.com/biz/prince-street-pizza-n...,3666,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 40.7229087705545, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '27 Prince St', 'address2': None,...",+12129664100,(212) 966-4100,1941.963817
2,WIhm0W9197f_rRtDziq5qQ,lombardis-pizza-new-york-4,Lombardi's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/UZ6V_h...,False,https://www.yelp.com/biz/lombardis-pizza-new-y...,6116,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7215934960083, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '32 Spring St', 'address2': '', '...",+12129417994,(212) 941-7994,1798.995978
3,C8j0q4Ma_S5hBGuAI-aaww,di-fara-pizza-brooklyn,Di Fara Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/4dfe9q...,False,https://www.yelp.com/biz/di-fara-pizza-brookly...,3519,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.625093, 'longitude': -73.961531}","[pickup, delivery]",$$,"{'address1': '1424 Ave J', 'address2': '', 'ad...",+17182581367,(718) 258-1367,9352.269073
4,EDYTUNp4bFO4dAwJMGuwCQ,pizza-collective-new-york,Pizza Collective,https://s3-media1.fl.yelpcdn.com/bphoto/kDJkUk...,False,https://www.yelp.com/biz/pizza-collective-new-...,31,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 40.77795, 'longitude': -73.98155}","[pickup, delivery]",NaN,"{'address1': '2060 Broadway', 'address2': None...",+16466930949,(646) 693-0949,8090.017092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7hEK78bm4KFkkltwIJz1gQ,grand-pizza-brooklyn,Grand Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/EbX65k...,False,https://www.yelp.com/biz/grand-pizza-brooklyn?...,4,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.71177, 'longitude': -73.94134}","[pickup, delivery]",$,"{'address1': '800 Grand St', 'address2': '', '...",,,4516.036576
996,m141OjVqd-VNhRrMBuW99A,zeffs-pizzeria-brooklyn,Zeff's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/Pnkjf6...,False,https://www.yelp.com/biz/zeffs-pizzeria-brookl...,18,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 40.7094993591309, 'longitude': -7...","[pickup, delivery]",$,"{'address1': '312 S 3rd St', 'address2': '', '...",+17185990545,(718) 599-0545,3349.689446
997,WHQpLvxkCK_BMtF8pIShnQ,carroll-place-new-york,Carroll Place,https://s3-media4.fl.yelpcdn.com/bphoto/_Qm18-...,False,https://www.yelp.com/biz/carroll-place-new-yor...,1088,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 40.7287, 'longitude': -73.99971}","[pickup, delivery]",$$,"{'address1': '157 Bleecker St', 'address2': ''...",+12122601700,(212) 260-1700,2628.319139
998,l1Hk9zSektC1YybmXZkfQg,the-new-taste-of-italy-jamaica,The New Taste of Italy,https://s3-media1.fl.yelpcdn.com/bphoto/-6UVkY...,False,https://www.yelp.com/biz/the-new-taste-of-ital...,5,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 40.6924462067728, 'longitude': -7...","[pickup, delivery]",$$,"{'address1': '84-07 Jamaica Ave', 'address2': ...",+17188058190,(718) 805-8190,11364.395599


In [49]:
headers = {'Authorization': f'Bearer {yelp_api_key}'}
url_params = {
                'term': 'pizza'.replace(' ', '+'),
                'location': 'NYC'.replace(' ', '+'),
                'limit': 50
            }
df = pd.DataFrame()
url_params['offset'] = 0
resp = requests.get('https://api.yelp.com/v3/businesses/search', headers=headers, params = url_params)
new_df = pd.DataFrame.from_dict(resp.json()['businesses'])
df.append(new_df)
df

""


In [60]:
df = pd.DataFrame({})
new_df = pd.DataFrame.from_dict(resp.json()['businesses'])
df = pd.concat(df, new_df)
df

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
# Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!